In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import math, random, sympy
import itertools

In [12]:
P_i = np.eye(2)
P_x = np.array([[0.,1.],[1.,0.]])
P_y = np.array([[0.,-1.j],[1.j,0.]])
P_z = np.array([[1.,0.],[0.,-1.]])

def read_txt(fname):
    lines = []
    nq = 0
    nt = 0
    
    with open(fname, 'r') as f:
        lines = f.readlines()
    if (len(lines) > 0):
        nq, nt = [int(s) for s in lines[0].split(',')]
    assert(len(lines) == nt+1)
    G_x = np.zeros((nq, nt), dtype=int)
    G_z = np.zeros((nq, nt), dtype=int)
    for i in range(nt):
        s = lines[i+1]
        #print(s)
        assert(len(s) == nq+1)
        for (j,c) in enumerate(s):
            if (c == 'X' or c == 'Y'):
                G_x[j][i] = True
            if (c == 'Z' or c == 'Y'):
                G_z[j][i] = True
    return G_x,G_z

def write_txt(fname, pauli_strings):
    nt = len(pauli_strings)
    if (nt > 0):
        nq = len(pauli_strings[0])
    else:
        nq = 0
    with open(fname, 'w+') as f:
        f.write('%d,%d\n' % (nq, nt))
        for p in pauli_strings:
            f.write('%s\n' % p)
    return nq, nt
    
def stab_to_matrix(row):
    # (x,z)
    nq = int(row.shape[1]/2)
    ps = []
    for i in range(nq):
        if (row[0,i] == 1):
            if (row[0,nq+i] == 1):
                ps.append(P_y)
            else:
                ps.append(P_x)
        else:
            if (row[0,nq+i] == 1):
                ps.append(P_z)
            else:
                ps.append(P_i)
    mat = ps[0]
    for j in range(len(ps)-1):
        mat = np.kron(mat, ps[j+1])
    return mat
    
def stab_dot(a, b):
    # a,b as rows (x,z)
    nq = int(a.shape[1]/2)
    assert(a.shape == (1, 2*nq) and b.shape == (1, 2*nq))
    res = a[:,:nq] @ b[:,nq:].T + a[:,nq:] @ b[:,:nq].T
    return res[0,0] % 2

def check_comm(i, tau, sub, nq):
    t = tau[i]
    res = True
    for j in range(len(sub)):
        sigx = np.zeros((1,nq*2))
        sigx[0,sub[j]] = 1 # (x,z)
        if (i==j):
            if (stab_dot(tau[i], sigx) == 0):
                return False
        else:
            if (stab_dot(tau[i], sigx) == 1):
                return False
    return True

def close_up_to_phase(U1, U2):
    return np.allclose(np.absolute(np.trace(U1 @ np.matrix(U2).getH())), 2) 
    
def tensor_decomp(U):
    if (U == None):
        print("Error: exceed recursive levels.")
        return None
    r, c = U.shape
    assert(r == c)
    if (r == 2):
        #print(U)
        if (close_up_to_phase(U, P_i)):
            return ('I', None)
        elif (close_up_to_phase(U, P_x)):
            return ('X', None)
        elif (close_up_to_phase(U, P_y)):
            return ('Y', None)
        elif (close_up_to_phase(U, P_z)):
            return ('Z', None)
        else:
            print("Error: Do not recognize base case.")
            return None
    elif (r > 2):
        side = int(r / 2)
        U00 = U[:side,:side]
        U01 = U[:side,side:]
        U10 = U[side:,:side]
        U11 = U[side:,side:]
        if (np.allclose(U01, np.zeros((side,side))) and np.allclose(U10, np.zeros((side,side)))):
            if (np.allclose(U00, U11)):
                return 'I', U00
            elif (np.allclose(U00, -U11)):
                return 'Z', U00
            else:
                print("Error: Do not recognize1.")
                return None
            
        elif (np.allclose(U00, np.zeros((side,side))) and np.allclose(U11, np.zeros((side,side)))):
            if (np.allclose(U01, U10)):
                return 'X', U10
            elif (np.allclose(U01, -U10)):
                return 'Y', -1.j*U10
            else:
                print("Error: Do not recognize2.")
                return None
            
        else:
            print("Error: Do not recognize3.")
            return None
        
def get_term(i, G_x, G_z):
    return np.array([np.concatenate((G_x[:,i], G_z[:,i]), axis=0).T])
    
def tapering(outfname, G_x, G_z):
    E = np.concatenate((G_z.T, G_x.T), axis=1)
    nt, nq = E.shape
    nq = int(nq/2)
    #print(E)
    ns = sympy.Matrix(E).nullspace()
    k = len(ns)
    if (k == 0):
        print("Nothing to taper.")
    tau = []
    for i in range(k):
        taui = np.array(ns[i]).astype(int)
        taui = np.mod(taui, np.full(taui.shape, 2))
        tau.append(taui.T) # as rows
    # Choose k qubits
    subs = list(itertools.combinations(range(nq), k))
    found = None
    for sub in subs:
        res = True
        for i in range(k):
            res = res and check_comm(i, tau, sub, nq)
        if (res):
            found = sub
    U = np.eye(2**nq)
    if (found != None or len(found) > 0):
        print("Hey! Found one possibility of tapering qubits: ")
        print(found)
        for i in range(k):
            sigx = np.zeros((1,nq*2), dtype=int)
            sigx[0,found[i]] = 1 # (x,z)
            Ui = 1./math.sqrt(2) * (stab_to_matrix(sigx) + stab_to_matrix(tau[i]))
            U = U @ Ui
    new_terms = []
    #print(U)
    for r in range(nt):
        new_term = ''
        V = np.matrix(U).getH() @ stab_to_matrix(get_term(r, G_x, G_z)) @ U
        for j in range(nq):
            p, V = tensor_decomp(V) # U = p tensor V
            new_term += p
        new_terms.append(new_term)
    print(new_terms)
    write_txt(outfname, new_terms)
    
    return U
            
def transform(infname, outfname):
    # Take a file with pauli strings,
    # Produce another file with new pauli strings
    # Return the transformation U applied to all strings
    G_x, G_z = read_txt(infname)
    U = tapering(outfname, G_x, G_z)
    return U

In [13]:
hydro = ['ZIII','IZII','IIZI','IIIZ','ZZII','ZIZI','ZIIZ','IZZI','IZIZ','IIZZ','YYXX','XYYX','YXXY','XXYY']
nq, nt = write_txt('taper_test.txt', hydro)
print(nq, nt)
transform('taper_test.txt', 'tapered_res.txt')

4 14
Hey! Found one possibility of tapering qubits: 
(1, 2, 3)
['ZIII', 'ZXII', 'ZIXI', 'ZIIX', 'IXII', 'IIXI', 'IIIX', 'IXXI', 'IXIX', 'IIXX', 'XIXX', 'XIIX', 'XXXI', 'XXII']


/software/Anaconda3-5.0.1-el7-x86_64/lib/python3.6/site-packages/ipykernel/__main__.py:86: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


array([[ 0.35355339,  0.35355339,  0.35355339,  0.35355339,  0.35355339,
         0.35355339,  0.35355339,  0.35355339,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.35355339, -0.35355339,  0.35355339, -0.35355339,  0.35355339,
        -0.35355339,  0.35355339, -0.35355339,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.35355339,  0.35355339, -0.35355339, -0.35355339,  0.35355339,
         0.35355339, -0.35355339, -0.35355339,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.35355339, -0.35355339, -0.35355339,  0.35355339,  0.35355339,
        -0.35355339, -0.35355339,  0.35355339,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.35355339,  0.35355339,  0

In [14]:
transform('sample.txt', 'sample_res.txt')

Nothing to taper.
Hey! Found one possibility of tapering qubits: 
()


/software/Anaconda3-5.0.1-el7-x86_64/lib/python3.6/site-packages/ipykernel/__main__.py:86: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


['IIIIIIII', 'XXYYIIII', 'XXYZZZZY', 'XXIXZZXI', 'XXIIYYII', 'XXIIIIYY', 'XYYXIIII', 'XYYZZZZX', 'XYIYZZXI', 'XYIIYXII', 'XYIIIIYX', 'XZXXZXII', 'XZXYZYII', 'XZXIXZXI', 'XZXIYZYI', 'XZXIIXZX', 'XZXIIYZY', 'XZYIYZXI', 'XZZXYZZY', 'XZZXIXXI', 'XZZYYZZX', 'XZZYIYXI', 'XZZZXIII', 'XZZZXZII', 'XZZZXIZI', 'XZZZXIIZ', 'XZZZZXYY', 'XZZZZYYX', 'XZZIXIII', 'XZIZXIII', 'XIZZXIII', 'YXXYIIII', 'YXXZZZZY', 'YXIXZZYI', 'YXIIXYII', 'YXIIIIXY', 'YYXXIIII', 'YYXZZZZX', 'YYIYZZYI', 'YYIIXXII', 'YYIIIIXX', 'YZXIXZYI', 'YZYXZXII', 'YZYYZYII', 'YZYIXZXI', 'YZYIYZYI', 'YZYIIXZX', 'YZYIIYZY', 'YZZXXZZY', 'YZZXIXYI', 'YZZYXZZX', 'YZZYIYYI', 'YZZZYIII', 'YZZZYZII', 'YZZZYIZI', 'YZZZYIIZ', 'YZZZZXXY', 'YZZZZYXX', 'YZZIYIII', 'YZIZYIII', 'YIZZYIII', 'ZIIIIIII', 'ZXZZZXII', 'ZYZZZYII', 'ZZIIIIII', 'ZIXZZZXI', 'ZIYZZZYI', 'ZIZIIIII', 'ZIIXZZZX', 'ZIIYZZZY', 'ZIIZIIII', 'ZIIIZIII', 'ZIIIIZII', 'ZIIIIIZI', 'ZIIIIIIZ', 'IXXYYIII', 'IXXIXZZX', 'IXXIIYYI', 'IXYYXIII', 'IXYIYZZX', 'IXYIIYXI', 'IXZXXZXI', 'IXZXYZYI', 'IX

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [15]:
transform('H2_6-31g_BKT_0.7_AS4.txt', 'H2_6-31g_BKT_0.7_AS4_res.txt')

Hey! Found one possibility of tapering qubits: 
(5, 7)


/software/Anaconda3-5.0.1-el7-x86_64/lib/python3.6/site-packages/ipykernel/__main__.py:86: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


['IIIIIIII', 'XYXYIIII', 'XXZYYXII', 'XYIXYIZX', 'XYIXZIYX', 'XYIYXIII', 'XXIYYXII', 'XXIYYXZI', 'XYIYIIXI', 'XYIZYIZI', 'XYIZZIYI', 'XXXIYIYI', 'XYYXZXII', 'XYYZZXIX', 'XXYIXIYI', 'XXYIYIXI', 'XYYIIXZI', 'XYZXYXII', 'XYZZYXIX', 'XYZIIXYI', 'XYIYXXII', 'XZXIIIII', 'XZXXIIIX', 'XZXZIIII', 'XZIIXIII', 'XZIIIIXI', 'XZYYIXZI', 'XZZYIXYI', 'XZIXIXXI', 'XZIZIXXX', 'XZIIXXII', 'YYYYIIII', 'YXZYXXII', 'YYIXXIZX', 'YYIXZIXX', 'YXIYXXII', 'YXIYXXZI', 'YYIYYIII', 'YYIYIIYI', 'YYIZXIZI', 'YYIZZIXI', 'YYXXZXII', 'YYXZZXIX', 'YXXIXIYI', 'YXXIYIXI', 'YYXIIXZI', 'YXYIXIXI', 'YYZXXXII', 'YYZZXXIX', 'YYZIIXXI', 'YYIYYXII', 'YZYIIIII', 'YZYXIIIX', 'YZYZIIII', 'YZIIYIII', 'YZIIIIYI', 'YZXYIXZI', 'YZZYIXXI', 'YZIXIXYI', 'YZIZIXYX', 'YZIIYXII', 'ZIIIIIII', 'ZYZYIIII', 'ZYIXXIYX', 'ZYIXYIXX', 'ZXIXIXZX', 'ZYIYZIII', 'ZYIYIIII', 'ZYIYIIZI', 'ZYIZXIYI', 'ZYIZYIXI', 'ZXIZIXZI', 'ZXIIZXIX', 'ZYXXYXII', 'ZYXZYXIX', 'ZYXIIXYI', 'ZYYXXXII', 'ZYYZXXIX', 'ZYYIIXXI', 'ZXZXIIII', 'ZXZZIIIX', 'ZYIYZXII', 'ZZIIIIII', 'ZI

array([[ 0.5,  0.5,  0. , ...,  0. ,  0. ,  0. ],
       [ 0.5, -0.5,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5, ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  0. , ..., -0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0.5,  0.5],
       [ 0. ,  0. ,  0. , ...,  0. ,  0.5, -0.5]])

In [ ]:
transform('H2O_6-31g_JW_104_AS6.txt', 'H2O_6-31g_JW_104_AS6_res.txt')

Hey! Found one possibility of tapering qubits: 
(5, 9)


/software/Anaconda3-5.0.1-el7-x86_64/lib/python3.6/site-packages/ipykernel/__main__.py:86: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
